# by *convention* __import__s should be the same

In [1]:
    ...; o = __name__ == '__main__'; ...;

Relative imports in the notebook are not allowed.

In [2]:
    if o:
        try: from . import relative
        except ImportError: assert True

This means _when a notebook is used as source_, relative imports have an unexpected behavior.

This notebook will `load_ipython_extension`s that allow relative imports by replacing them with the `blank` convention below ⬇️

In [3]:
    blank = """try: 
        from . import object
    except: 
        import object"""

---
#### parent __import__s

> It is not possible to __import__ a notebook above the current directory so there no reason to implement higher-level __import__s.

In [14]:
    if o:
        try: from .. import object
        except ValueError as Exception: assert Exception.args == (
            'attempted relative import beyond top-level package',)

---

## The `Relative` `ast.NodeTransformer`

In [5]:
    from ast import parse, NodeTransformer, copy_location
    class Relative(NodeTransformer):
        """Replace __import__ when {node.level is 1} with the `try` statement in {blank}."""
        blank = parse(blank).body[0]
        def visit_ImportFrom(Relative, node):
            if node.module : return node
            if node.level is 1:
                Relative.blank.body[0].names = \
                Relative.blank.handlers[0].body[0].names = node.names
                return copy_location(Relative.blank, node)
            return node

## The Extensions

    %unload rites.relative

In [6]:
    def unload_ipython_extension(ip=get_ipython()): return [
        object for object in ip.ast_transformers if not isinstance(object, Relative)]

    %reload rites.relative

In [7]:
    def load_ipython_extension(ip=get_ipython()): ip.ast_transformers = unload_ipython_extension(ip) +[Relative()]

## Developer

### Testing

Test the `rites.relative` extension on itself; remove any python files for the test.

In [8]:
    if o:
        %reload_ext rites
        load_ipython_extension()
        %rm relative.py

rm: relative.py: No such file or directory


In [12]:
    from . import relative

    __test__ = dict(
        imports=""">>> assert relative is relative.relative""",
        file=""">>> assert relative.__file__.endswith('.ipynb')""",
        type=""">>> assert isinstance(relative, __import__('types').ModuleType)""")

### Tear Down

> Export the `relative.ipynb` as a python script because it does not rely on anything from `rites`.

In [13]:
    if o:
        __import__('doctest').testmod(verbose=2)
        !jupyter nbconvert --to python relative.ipynb

Trying:
    assert relative.__file__.endswith('.ipynb')
Expecting nothing
ok
Trying:
    assert relative is relative.relative
Expecting nothing
ok
Trying:
    assert isinstance(relative, __import__('types').ModuleType)
Expecting nothing
ok
5 items had no tests:
    __main__
    __main__.Relative
    __main__.Relative.visit_ImportFrom
    __main__.load_ipython_extension
    __main__.unload_ipython_extension
3 items passed all tests:
   1 tests in __main__.__test__.file
   1 tests in __main__.__test__.imports
   1 tests in __main__.__test__.type
3 tests in 8 items.
3 passed and 0 failed.
Test passed.
[NbConvertApp] Converting notebook relative.ipynb to python
[NbConvertApp] Writing 2642 bytes to relative.py
